## Part 6: Transforming Data

In this section we'll see two data transformation techinques:
- Sorting lets you order your data to highlight top values, see trends, or restore a logical sequence.
- Most real-world analysis centers on **grouping** observations and computing **aggregate** statistics (mean, median, counts, etc.) rather than peeking at individual rows.

### 1. Sorting (small DataFrame)

In [ ]:
import pandas as pd

people = {
    "first": ["Alice", "Bob", "Carol", "Sarah"],
    "last": ["Smith", "Jones", "Lee", "Jones"],
    "email": [
        "alice@example.com",
        "bob@example.com",
        "carol@example.com",
        "sarah@example.com",
    ],
    "uid": ["AS100293", "BJ240806", "CL150510", "SJ251203"],
    "year": [1993, 2006, 2010, 2003],
    "age": [32, 19, 15, 22],
}
df_small = pd.DataFrame(people)

In [ ]:
# 1a. Sort by a single column (year ascending by default)
df_small = df_small.sort_values(by="year")
print("Sorted by year (oldest → youngest):")
print(df_small)

# 1b. Sort descending
df_small = df_small.sort_values(by="year", ascending=False)
print("\nSorted by year descending (youngest → oldest):")
print(df_small)

# 1c. Add an extra person to illustrate multi-column sort
df_small = df_small.append(
    {"first": "Tony", "last": "Stark", "year": 2000},
    ignore_index=True,
)
print("\nAfter adding Tony Stark:")
print(df_small)

# 1d. Sort by last name descending, then year ascending
df_small = df_small.sort_values(
    by=["last", "year"],
    ascending=[False, True]
)
print("\nSorted by last ↓, then year ↑:")
print(df_small)

# 1e. Restore the original index order
df_small = df_small.sort_index()
print("\nRestored original index with sort_index():")
print(df_small)

# 1f. Sorting a Series
print("\nYear series sorted:")
print(df_small["year"].sort_values())

### 2. Sorting (big DataFrame)

In [ ]:
# 2a. Sort by Country alphabetically
df = df.sort_values(by="Country")
print("First 50 respondents sorted by Country:")
print(df.head(50))

# 2b. Sort by Country then by Salary (low → high)
df = df.sort_values(by=["Country", "Salary"])
print("\nSample sorted by Country then Salary:")
print(df.head(10))

# 2c. Top 10 highest salaries
top10_salary = df.nlargest(10, "Salary")
print("\nTop 10 salaries:")
print(top10_salary[["Country", "Salary"]])

# 2d. Bottom 10 lowest salaries
bottom10_salary = df.nsmallest(10, "Salary")
print("\nBottom 10 salaries:")
print(bottom10_salary[["Country", "Salary"]])

### 3. Basic aggregation on one column

In [ ]:
# Median salary across all respondents
print("Global median salary:", df["Salary"].median())

# What if you call df.median()?
print("\nAll numeric-column medians:")
df.median(numeric_only=True)

> **Note:** `.median()` ignores NaNs.  Calling `df.median(numeric_only=True)` returns a Series of medians for every numeric column.

### 4. Quick summary with `.describe()`

In [ ]:
# Quick summary with `.describe()` on the entire DataFrame
df.describe()

# Quick summary of the "Salary" column
df["Salary"].describe()

* **count**: number of non‐missing values
* **mean**: average (sensitive to outliers)
* **50%** quantile = **median**
* **std**, **min**, **max**, and other percentiles

In [ ]:
# Compare count vs. value_counts
print("Non-null Salary count:", df["Salary"].count())
print("Value counts of a categorical column (e.g. Age):")
print(df["Age"].value_counts().head())

# Example: AISelect usage
print("\nAISelect question (code lookup):")
print(schema_df.loc["AISelect", "question"])

print("\nResponse counts for AISelect:")
print(df["AISelect"].value_counts(dropna=False))

print("\nRelative frequencies of AISelect:")
print(df["AISelect"].value_counts(normalize=True).head())

### 5. GroupBy by country

In [ ]:
# Quick alternative to get country counts:
print(df["Country"].value_counts().head())

# The GroupBy object
country_grp = df.groupby("Country")

# You can pull out a single country’s subframe:
swiss = country_grp.get_group("Switzerland")
print("\nFirst rows for Switzerland:")
print(swiss.head(3))

# This is equivalent to a mask:
mask = df["Country"] == "Switzerland"
print("\nSame via boolean mask:")
print(df.loc[mask].head(3))

### 6. Aggregating within groups

#### 6.1 Value counts of AISelect by country

In [ ]:
# AISelect counts per country (multi‐indexed Series)
ai_by_ctry = country_grp["AISelect"].value_counts()
print(ai_by_ctry.head(10))

# To select Switzerland’s AISelect breakdown:
print("\nAISelect counts in Switzerland:")
print(ai_by_ctry.loc["Switzerland"])

#### 6.2 Median salary per country

In [ ]:
med_salary_by_ctry = country_grp["Salary"].median()
print("\nMedian salary by country:")
print(med_salary_by_ctry.head(10))

# Specific country:
print("\nSwitzerland median salary:", med_salary_by_ctry.loc["Switzerland"])

#### 6.3 Multiple aggregations with `.agg()`

In [ ]:
# Compute both mean and median salary by country
salary_stats = country_grp["Salary"].agg(["mean", "median"]).sort_values("median", ascending=False)
print("\nSalary mean & median by country:")
print(salary_stats.head(10))

#### 6.4 Grouping Gotcha: Summing Boolean Masks by Group

In [ ]:
# Mask for Python users in India
filt = df["Country"] == "India"
df.loc[filt, "LanguageWorkedWith"].str.contains("Python", na=False)

you get a Series of `True`/`False` for India, and can do:

In [ ]:
(df.loc[filt, "LanguageWorkedWith"].str.contains("Python", na=False)).sum()

to count them. But if you try directly on the `GroupBy`:

In [ ]:
country_grp["LanguageWorkedWith"].str.contains("Python", na=False).sum()

you’ll get an `AttributeError`, because `country_grp["..."]` is a grouped object, not a Series.

In [ ]:
country_grp["LanguageWorkedWith"].apply(lambda s: s.str.contains("Python", na=False).sum())

#### Together Exercises
> **How to compute, for each country, the percentage of respondents who know Python?**

In [ ]:
# 1. Number of respondents per country
country_respondents = df["Country"].value_counts()  


# 2. Number of Python users per country
country_knows_python = country_grp["LanguageWorkedWith"].apply(lambda s: s.str.contains("Python", na=False).sum())

# 3. Combine into one DataFrame
python_stats = pd.concat(
      [country_respondents, country_knows_python],
      axis="columns",
      sort=False,
)
python_stats.columns = ["NumRespondents", "NumKnowsPython"]

# 4. Compute percentage
python_stats["PctKnowsPython"] = (
      python_stats["NumKnowsPython"] / python_stats["NumRespondents"] * 100
)

# 5. Sort and display
python_stats = python_stats.sort_values("PctKnowsPython", ascending=False)
python_stats.head(10)

or:

In [ ]:
country_grp['LanguageWorkedWith'].apply( lambda x: x.str.contains('Python').sum() / len(x) * 100 )

### Exercises for Part 6

#### Exercise 6.1
- For each country in the list `["Switzerland", "Germany", "United States of America", "China", "France", "Italy"]`, compute:
  - The percentage of respondents that answered “Yes” to `AISelect`.
  - The median `Salary`.
- Present the results in a DataFrame with columns:  
  `Country | PctAIUsers | AvgSalary`.

#### Exercise 6.2
1. **Step 1**
   - Compute the **mean** and **median** `Salary` for each country.  
   - Store the result in a DataFrame with columns: `Country | MeanSalary | MedianSalary | NRespondents`.
2. **Step 2**  
   - Restrict to the **top 10 countries by** `NRespondents`.  
   - Reset the index so `Country` is a column.

#### Exercise 6.3
1. **Step 1**  
   - Using the existing `"Age"` column (e.g. `"25-34 years old"`), group respondents by bracket.  
   - For each bracket compute:
     - `AvgSalary` (mean `Salary`),  
     - `PctKnowsPython` (percentage who know Python, based on your boolean column).
     - Create a DataFrame:  
     ```
     AgeBracket | AvgSalary | PctKnowsPython | NRespondents
     ```
2. **Step 2**  
   - Count how many respondents fall into each `"Age"` bracket.